In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24708151
paper_name = 'schlecht_stonge_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [20]:
original_data = pd.read_excel('raw_data/Table.S1.xlsx', sheet_name='Table S1')

In [21]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5050 x 17


In [22]:
original_data.head()

,Identifier,Gene,ORF,HD,RD,ENH,DIM,YPD (CRS),YPD (q-value),YPE (CRS),YPE (q-value),YPG (CRS),YPG (q-value),YPL (CRS),YPL (q-value),YPE* (CRS),YPE* (q-value)
0,YAL002W::chr1_1,VPS8,YAL002W,Y,NaN,NaN,NaN,-0.209411,0.575205,-0.184952,0.486516,-0.319782,0.273658,-0.267636,0.385426,NaN,NaN
1,YAL004W::chr1_1,YAL004W,YAL004W,Y,NaN,NaN,NaN,-0.197994,0.451079,0.220628,0.190319,-0.049425,0.753078,0.020632,0.885515,NaN,NaN
2,YAL005C::chr1_1,SSA1,YAL005C,Y,NaN,NaN,NaN,0.019326,0.853493,0.091281,0.389141,0.086033,0.446926,-0.001708,0.982579,NaN,NaN
3,YAL007C::chr1_1,ERP2,YAL007C,Y,NaN,NaN,NaN,0.121335,0.493788,0.151073,0.198637,0.120506,0.332264,0.117534,0.368390,NaN,NaN
4,YAL008W::chr1_1,FUN14,YAL008W,Y,NaN,NaN,NaN,0.122860,0.643783,-0.177721,0.382949,0.195369,0.374872,-0.080636,0.702752,NaN,NaN


In [23]:
original_data['orf'] = original_data['ORF'].astype(str)

In [24]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [25]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Identifier, Gene, ORF, HD, RD, ENH, DIM, YPD (CRS), YPD (q-value), YPE (CRS), YPE (q-value), YPG (CRS), YPG (q-value), YPL (CRS), YPL (q-value), YPE* (CRS), YPE* (q-value), orf]
Index: []


In [27]:
original_data.set_index('orf', inplace=True)

In [28]:
data_cols = [c for c in original_data.columns.values if '(CRS)' in c and 'YPE*' not in c]

In [29]:
data_cols

['YPD (CRS)', 'YPE (CRS)', 'YPG (CRS)', 'YPL (CRS)']

In [30]:
original_data = original_data[data_cols].copy()

In [31]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data.shape

(4882, 4)

# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = [692, 453, 690, 691]
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,692,453,690,691
data_type,value,value,value,value
orf,,,,
YAL002W,-0.209411,-0.184952,-0.319782,-0.267636
YAL004W,-0.197994,0.220628,-0.049425,0.020632
YAL005C,0.019326,0.091281,0.086033,-0.001708
YAL007C,0.121335,0.151073,0.120506,0.117534
YAL008W,0.122860,-0.177721,0.195369,-0.080636


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,692,453,690,691
,data_type,value,value,value,value
gene_id,orf,,,,
2,YAL002W,-0.209411,-0.184952,-0.319782,-0.267636
1863,YAL004W,-0.197994,0.220628,-0.049425,0.020632
4,YAL005C,0.019326,0.091281,0.086033,-0.001708
5,YAL007C,0.121335,0.151073,0.120506,0.117534
6,YAL008W,0.122860,-0.177721,0.195369,-0.080636


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,692,453,690,691,692,453,690,691
,data_type,value,value,value,value,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,
2,YAL002W,-0.209411,-0.184952,-0.319782,-0.267636,-0.858381,-0.495023,-0.771995,-0.508150
1863,YAL004W,-0.197994,0.220628,-0.049425,0.020632,-0.826809,0.297433,-0.214839,0.052529
4,YAL005C,0.019326,0.091281,0.086033,-0.001708,-0.225871,0.044702,0.064315,0.009077
5,YAL007C,0.121335,0.151073,0.120506,0.117534,0.056205,0.161529,0.135359,0.241002
6,YAL008W,0.122860,-0.177721,0.195369,-0.080636,0.060422,-0.480895,0.289637,-0.144437


# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [45]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/4 [00:00<?, ?it/s]

Deleting all datasets for PMID 24708151...
Inserting the new data...


100%|██████████| 4/4 [00:30<00:00,  7.61s/it]

Updating the data_modified_on field...
